# swmm_api basic tutorial

In this Jupyter Notebook, our goal is to explore basic functionalities of `swmm_api`. You can find the documentation [here](https://markuspichler.gitlab.io/swmm_api/README.html#).

For our analyses, we will use Example2-Post.inp, that we worked on in class. Verify your code by comparing to the .inp file

# 1. Reading the INP file

In [ ]:
import swmm_api
print(f'{swmm_api.__version__ = }')

### Import packages

In [ ]:
from swmm_api.input_file import read_inp_file, SwmmInput, section_labels as sections
from swmm_api.input_file.sections import Outfall

In [ ]:
# read the file 
# the inp object is like a dictionary

inp = read_inp_file('Example2-Post.inp') 

### Explore different sections

In [ ]:
# get the sections
inp.keys()

In [ ]:
# get different settings in a specific section
# for example OPTIONS

inp.OPTIONS

# you can also access it like this
# inp[sections.OPTIONS]

In [ ]:
# to convert a section to a string, use the .to_inp_lines() method

print(inp[sections.OPTIONS].to_inp_lines())

In [ ]:
# SUBCATCHMENTS

print(inp[sections.SUBCATCHMENTS].to_inp_lines())

In [ ]:
# for table-like structures, you can convert a section to a pandas dataframe (not for OPTIONS, REPORTS...)

inp.SUBCATCHMENTS.frame

### Explore a single section

In [ ]:
# for example CONDUITS

print(inp.CONDUITS.to_inp_lines())

In [ ]:
# explore C1

inp.CONDUITS['C1']

In [ ]:
# get C1 length and roughness 

print(inp.CONDUITS['C1'].length)
print(inp.CONDUITS['C1'].roughness)

In [ ]:
# RAINGAGES section

inp.RAINGAGES

In [ ]:
# get specific attribute

timeseries_name = inp.RAINGAGES['RainGage'].timeseries
print(timeseries_name)

In [ ]:
# get TIMESERIES

inp.TIMESERIES[timeseries_name].frame

# Exercise:
1. get the shape and the diameter of link C11
2. get the elevation of junction J6
3. get the flow routing method

# 2. Plotting the model

import classes

In [ ]:
from swmm_api import SwmmInput
from swmm_api.input_file.macros import nodes_dict, PlottingMap, init_empty_map_plot, plot_longitudinal
from swmm_api.input_file.macros import add_node_map, add_link_map, add_subcatchment_map, add_node_labels, plot_map

In [ ]:
# plot the model
# default

inp = read_inp_file('Example2-Post.inp') 
fig, ax = plot_map(inp)

In [ ]:
# make it prettier

m = PlottingMap(inp).add_subcatchment_map(add_connector_arrows=True).add_link_map(add_arrows=True).add_node_map().add_node_labels(y_offset=80)
# fig, ax = m.fig, m.ax

In [ ]:
# profile plot

_, ax = plot_longitudinal(inp, start_node='J1', end_node='J10')
ax.set_title("start_node='J1', end_node='J10'")

# 3. Running a simulation and extracting results

import classes

In [ ]:
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.run_swmm import swmm5_run_epa, swmm5_run_progress
from swmm_api.run_swmm.run_temporary import swmm5_run_temporary

In [ ]:
inp = read_inp_file('Example2-Post.inp') 

## 3.1. Running the simulation
### Note:

setting `run = swmm5_run_progress` will run pyswmm (swmm5.dll)

setting `run = swmm5_run_epa` will run swmm5.exe

see more info https://markuspichler.gitlab.io/swmm_api/api_reference/run.html

In [ ]:
# run simulation

with swmm5_run_temporary(inp.copy(), run = swmm5_run_progress, label='example_run_swmm') as res:
    out = res.out  # type: SwmmOutput
    rpt = res.rpt  # type: SwmmReport

## 3.2 Reading the RPT file

### What data is available in the RPT file?

see all options here https://markuspichler.gitlab.io/swmm_api/api_reference/rpt/swmm_api.SwmmReport.html#swmm_api.SwmmReport

In [ ]:
# get available sections

rpt.available_parts

In [ ]:
# print individual parts
# example Runoff Quantity Continuity

rpt.print_raw_part('Runoff Quantity Continuity')

In [ ]:
# print Analysis Options

print(rpt._raw_parts['Analysis Options'])

In [ ]:
# get just the flow units

rpt.analysis_options['Flow Units']

In [ ]:
# get node flooding summary

print(rpt.node_flooding_summary)

In [ ]:
# get conduit surchange summary

print(rpt.conduit_surcharge_summary)

# Exercise:
1. get the link flow summary
2. get the maximum flow for C11
3. get the flow capacity for C11
4. get the maximum total inflow for J1

## 3.3 Example for reading OUT file
see all classess and attributes https://markuspichler.gitlab.io/swmm_api/api_reference/out/swmm_api.SwmmOutput.html

### Which variables are available in the OUT file for each object type?

In [ ]:
out.variables

### What are the object labels?

In [ ]:
out.labels

In [ ]:
out.model_properties

## 3.4 Retrieving the data


In [ ]:
from swmm_api.output_file import OBJECTS, VARIABLES

### Nodes
https://markuspichler.gitlab.io/swmm_api/api_reference/out/swmm_api.output_file.definitions.NODE_VARIABLES.html#swmm_api.output_file.definitions.NODE_VARIABLES

In [ ]:
# to get all data of a node
out.get_part(OBJECTS.NODE, 'J1')

In [ ]:
# get specific variable
out.get_part(OBJECTS.NODE, 'J1', VARIABLES.NODE.HEAD).to_frame()

In [ ]:
# Reading only a slice of the results timeseries

import pandas as pd

custom_start = pd.Timestamp('2007-01-01 08:00')
custom_end = pd.Timestamp('2007-01-01 10:00')

In [ ]:
# custom_start

out.get_part(OBJECTS.NODE, 'J1', VARIABLES.NODE.HEAD, start=custom_start)

In [ ]:
# custom_end

out.get_part(OBJECTS.NODE, 'J1', VARIABLES.NODE.HEAD, end=custom_end)

In [ ]:
# custom_start_end

out.get_part(OBJECTS.NODE, 'J1', VARIABLES.NODE.HEAD, start=custom_start, end=custom_end)

### Links
https://markuspichler.gitlab.io/swmm_api/api_reference/out/swmm_api.output_file.definitions.LINK_VARIABLES.html

In [ ]:
# to get all data of a node
out.get_part(OBJECTS.LINK, 'C1')

In [ ]:
# get specific variable
out.get_part(OBJECTS.LINK, 'C1', VARIABLES.LINK.FLOW).to_frame()

## 3.5 Convert data
You can convert the data to dataframe or numpy array. This is useful to store and manipulate it to your specific application.

In [ ]:
# convert all data to in numpy array format
type(out.to_numpy())

In [ ]:
# retrieve data in Pandas dataframe format
out.to_frame()

# 4. Plotting results
Select some output time series variables, such as subcatchment runoff, link flow, outfall total inflow to plot

## 4.1 Timeseries plots

In [ ]:
# plot system outflow and inflow
# system outflow = total inflow at O1
# system inflow = system runoff

import matplotlib.pyplot as plt

out.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).plot()
out.get_part(OBJECTS.SYSTEM, None, VARIABLES.SYSTEM.RUNOFF).plot()
plt.xlabel('Time (hours)')
plt.ylabel('CFS')
plt.title('System inflow/outflow')
plt.legend()
plt.grid(True)

In [ ]:
# plot runoff from individual subcatchments

out.get_part(OBJECTS.SUBCATCHMENT, 'S1', VARIABLES.SUBCATCHMENT.RUNOFF).plot()
out.get_part(OBJECTS.SUBCATCHMENT, 'S2', VARIABLES.SUBCATCHMENT.RUNOFF).plot()
out.get_part(OBJECTS.SUBCATCHMENT, 'S3', VARIABLES.SUBCATCHMENT.RUNOFF).plot()

plt.xlabel('Time (hours)')
plt.ylabel('Runoff CFS')
plt.title('Runoff')
plt.legend()


In [ ]:
out.get_part(OBJECTS.SYSTEM, None, VARIABLES.SYSTEM.FLOODING).plot()

In [ ]:
out.index

## Exercise:
1. choose three conduits and plot the flow on the same plot
2. choose three junctions and plot the depth
3. make another timeseries plot

## 4.2 Spatial plots

In [ ]:
from swmm_api.input_file.macros.plotting_map import *

link_flow = out.get_part(OBJECTS.LINK, None, VARIABLES.LINK.FLOW)
node_flooding = out.get_part(OBJECTS.NODE, None, VARIABLES.NODE.FLOODING)
sc_runoff = out.get_part(OBJECTS.SUBCATCHMENT, None, VARIABLES.SUBCATCHMENT.RUNOFF)

time = '2007-01-01 00:03:00'

_, ax = init_empty_map_plot()
add_link_map(ax, inp, values_dict=link_flow.loc[time].to_dict(), make_width_proportional=True, cmap='YlGn', colorbar_kwargs={'label':'Link Flow'})
add_subcatchment_map(ax, inp, values_dict=sc_runoff.loc[time].to_dict(), cmap='BuPu', colorbar_kwargs={'label':'SC Runoff'})
add_node_map(ax, inp, make_size_proportional=True, cmap='Reds', values_dict=node_flooding.loc[time].to_dict(), colorbar_kwargs={'label':'Node Flooding'})
add_node_labels(ax, inp, size=5, y_offset=30)
ax.set_title(f'Time: {time}')

In [ ]:
from swmm_api.input_file.macros.plotting_map import init_empty_map_plot, add_subcatchments_labels, add_node_labels, add_link_map, add_node_map, add_subcatchment_map
import pandas as pd

sc_runoff = rpt.subcatchment_runoff_summary[f'Total_Runoff_10^6 {rpt.unit.VOL2}']
node_flooded_hrs = rpt.node_inflow_summary['Time of Max_Occurrence_days hr:min'] / pd.Timedelta(minutes=1)
link_full = rpt.link_flow_summary['Max/_Full_Flow']

_, ax = init_empty_map_plot()
add_link_map(ax, inp, values_dict=link_full.to_dict(), make_width_proportional=False, cmap='YlGn', colorbar_kwargs={'label':'Link Ratio Max Full in %'}, add_arrows=True)
add_subcatchment_map(ax, inp, values_dict=sc_runoff.to_dict(), cmap='YlGn', colorbar_kwargs={'label':f'SC Total Runoff in 10^6 {rpt.unit.VOL2}'})
add_node_map(ax, inp, make_size_proportional=True, cmap='Reds', values_dict=node_flooded_hrs.to_dict(), colorbar_kwargs={'label':'Node time max occurence in minutes'})
add_node_labels(ax, inp, size=5, y_offset=30)
add_subcatchments_labels(ax, inp, size=5, y_offset=40, color='black')